In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
seed = 1
batch_size = 64
test_batch_size = 64
momentum = 0.9
epochs = 50
log_interval = 100
lr = 0.01
save_model = False
num_classes = 10
criterion = nn.CrossEntropyLoss()
torch.backends.cudnn.benchmark=True


In [3]:
class CNNNet(nn.Module):
    def __init__(self):
        super(CNNNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5, 1)
        self.conv2 = nn.Conv2d(64, 256, 5, 1)
        self.fc1= nn.Linear(6400, 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.dropout1 = nn.Dropout2d(0.2)
        self.dropout2 = nn.Dropout2d(0.2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = self.dropout1(x)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        return F.relu(self.fc2(x))

In [4]:
def train(model, device, train_loader, optimizer, log_interval, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [5]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).sum().item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
use_cuda =  torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}

torch.manual_seed(seed)


train_dataset=datasets.CIFAR10('./data_c', train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
#                       transforms.Normalize((0.1307,), (0.3081,))
                       
                   ]))

test_dataset=datasets.CIFAR10('./data_c', train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
#                       transforms.Normalize((0.1307,), (0.3081,))
    ]))

image, label = train_dataset[0]
print(image.size())
print(label)

Files already downloaded and verified
Files already downloaded and verified
torch.Size([3, 32, 32])
6


In [7]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=test_batch_size, shuffle=False, num_workers=2)


In [8]:
for images, labels in train_loader:
    print(images.size())
    print(images[0].size())
    print(labels.size())
    
    break

torch.Size([64, 3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([64])


In [9]:
model = CNNNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=5e-4)

def run():
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, log_interval, epoch)
        test(model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"mlp.pt")

In [10]:
%%time

run()

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.303777
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.258908
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.133685
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.922502
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.780693
Train Epoch: 1 [32000/50000 (64%)]	Loss: 1.632212
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.646587
Train Epoch: 1 [44800/50000 (90%)]	Loss: 1.676590

Test set: Average loss: 0.0227, Accuracy: 4816/10000 (48%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 1.592931
Train Epoch: 2 [6400/50000 (13%)]	Loss: 1.523198
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.304960
Train Epoch: 2 [19200/50000 (38%)]	Loss: 1.285889
Train Epoch: 2 [25600/50000 (51%)]	Loss: 1.173870
Train Epoch: 2 [32000/50000 (64%)]	Loss: 1.429218
Train Epoch: 2 [38400/50000 (77%)]	Loss: 1.222576
Train Epoch: 2 [44800/50000 (90%)]	Loss: 1.266640

Test set: Average loss: 0.0190, Accuracy: 5720/10000 (57%)

Train Epoch: 3 [0/50000 (0%)]	Loss: 1.089835
Train Epoch: 3 [6400/50000 (13%)]	Loss: 1.263


Test set: Average loss: 0.0149, Accuracy: 7297/10000 (73%)

Train Epoch: 19 [0/50000 (0%)]	Loss: 0.186260
Train Epoch: 19 [6400/50000 (13%)]	Loss: 0.144150
Train Epoch: 19 [12800/50000 (26%)]	Loss: 0.163894
Train Epoch: 19 [19200/50000 (38%)]	Loss: 0.195776
Train Epoch: 19 [25600/50000 (51%)]	Loss: 0.313050
Train Epoch: 19 [32000/50000 (64%)]	Loss: 0.142444
Train Epoch: 19 [38400/50000 (77%)]	Loss: 0.247951
Train Epoch: 19 [44800/50000 (90%)]	Loss: 0.306024

Test set: Average loss: 0.0152, Accuracy: 7283/10000 (73%)

Train Epoch: 20 [0/50000 (0%)]	Loss: 0.256317
Train Epoch: 20 [6400/50000 (13%)]	Loss: 0.133042
Train Epoch: 20 [12800/50000 (26%)]	Loss: 0.287041
Train Epoch: 20 [19200/50000 (38%)]	Loss: 0.233262
Train Epoch: 20 [25600/50000 (51%)]	Loss: 0.176284
Train Epoch: 20 [32000/50000 (64%)]	Loss: 0.258778
Train Epoch: 20 [38400/50000 (77%)]	Loss: 0.084196
Train Epoch: 20 [44800/50000 (90%)]	Loss: 0.107943

Test set: Average loss: 0.0153, Accuracy: 7317/10000 (73%)

Train Epoch: 

Train Epoch: 36 [38400/50000 (77%)]	Loss: 0.035533
Train Epoch: 36 [44800/50000 (90%)]	Loss: 0.119859

Test set: Average loss: 0.0158, Accuracy: 7390/10000 (74%)

Train Epoch: 37 [0/50000 (0%)]	Loss: 0.181763
Train Epoch: 37 [6400/50000 (13%)]	Loss: 0.053019
Train Epoch: 37 [12800/50000 (26%)]	Loss: 0.117068
Train Epoch: 37 [19200/50000 (38%)]	Loss: 0.085818
Train Epoch: 37 [25600/50000 (51%)]	Loss: 0.085321
Train Epoch: 37 [32000/50000 (64%)]	Loss: 0.065482
Train Epoch: 37 [38400/50000 (77%)]	Loss: 0.063739
Train Epoch: 37 [44800/50000 (90%)]	Loss: 0.091011

Test set: Average loss: 0.0159, Accuracy: 7380/10000 (74%)

Train Epoch: 38 [0/50000 (0%)]	Loss: 0.172716
Train Epoch: 38 [6400/50000 (13%)]	Loss: 0.117264
Train Epoch: 38 [12800/50000 (26%)]	Loss: 0.065315
Train Epoch: 38 [19200/50000 (38%)]	Loss: 0.049378
Train Epoch: 38 [25600/50000 (51%)]	Loss: 0.091719
Train Epoch: 38 [32000/50000 (64%)]	Loss: 0.045092
Train Epoch: 38 [38400/50000 (77%)]	Loss: 0.053437
Train Epoch: 38 [44800/